In [1]:
import pandas as pd
import torch
from torch_rechub.models.multi_task import single_task
from torch_rechub.trainers import MTLTrainer
from torch_rechub.basic.features import DenseFeature, SparseFeature
from torch_rechub.utils.data import DataGenerator

In [ ]:
model_name = 'single_task'
epoch = 100  
learning_rate = 1e-3
batch_size = 1024
weight_decay = 1e-6
save_dir = './save_dir'
seed = 2024
torch.manual_seed(seed)

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

In [4]:
task_select = 0  #0: cvr, 1:ctr
def get_census_data_dict(model_name, task_select = task_select, data_path="data/census-income"):
    df_train = pd.read_csv(data_path + '/census_income_train.csv')
    df_val = pd.read_csv(data_path + '/census_income_val.csv')
    df_test = pd.read_csv(data_path + '/census_income_test.csv')
    print("train : val : test = %d %d %d" % (len(df_train), len(df_val), len(df_test)))
    train_idx, val_idx = df_train.shape[0], df_train.shape[0] + df_val.shape[0]
    data = pd.concat([df_train, df_val, df_test], axis=0)
    data = data.fillna(0)
    #task 1 (as cvr): main task, income prediction
    #task 2(as ctr): auxiliary task, marital status prediction
    data.rename(columns={'income': 'cvr_label', 'marital status': 'ctr_label'}, inplace=True)
    data["ctcvr_label"] = data['cvr_label'] * data['ctr_label']

    col_names = data.columns.values.tolist()
    dense_cols = ['age', 'wage per hour', 'capital gains', 'capital losses', 'divdends from stocks', 'num persons worked for employer', 'weeks worked in year']
    sparse_cols = [col for col in col_names if col not in dense_cols and col not in ['cvr_label', 'ctr_label', 'ctcvr_label']]
    print("sparse cols:%d dense cols:%d" % (len(sparse_cols), len(dense_cols)))
    #define dense and sparse features
    if model_name == "ESMM":
        label_cols = ['cvr_label', 'ctr_label', "ctcvr_label"]  #the order of 3 labels must fixed as this
        #ESMM only for sparse features in origin paper
        user_cols = ['industry code', 'occupation code', 'race', 'education', 'sex']  #assumption features split for user and item
        item_cols = [col for col in sparse_cols if col not in user_cols]
        user_features = [SparseFeature(col, data[col].max() + 1, embed_dim=16) for col in user_cols]
        item_features = [SparseFeature(col, data[col].max() + 1, embed_dim=16) for col in item_cols]
        x_train, y_train = {name: data[name].values[:train_idx] for name in sparse_cols}, data[label_cols].values[:train_idx]
        x_val, y_val = {name: data[name].values[train_idx:val_idx] for name in sparse_cols}, data[label_cols].values[train_idx:val_idx]
        x_test, y_test = {name: data[name].values[val_idx:] for name in sparse_cols}, data[label_cols].values[val_idx:]
        return user_features, item_features, x_train, y_train, x_val, y_val, x_test, y_test
    else:
        label_cols = ['cvr_label', 'ctr_label']  #the order of labels can be any
        
        label_col = [label_cols[task_select]]
        used_cols = sparse_cols + dense_cols
        features = [SparseFeature(col, data[col].max()+1, embed_dim=4)for col in sparse_cols] \
                   + [DenseFeature(col) for col in dense_cols]
        x_train, y_train = {name: data[name].values[:train_idx] for name in used_cols}, data[label_col].values[:train_idx]
        x_val, y_val = {name: data[name].values[train_idx:val_idx] for name in used_cols}, data[label_col].values[train_idx:val_idx]
        x_test, y_test = {name: data[name].values[val_idx:] for name in used_cols}, data[label_col].values[val_idx:]
        return features, x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
if model_name == "single_task":
    task_type = "classification"
    features, x_train, y_train, x_val, y_val, x_test, y_test = get_census_data_dict(model_name)
    # Set model hyperparameters
    model = single_task(features, task_type, hidden_units = {"dims": [16,8]}, tower_params_list = {"dims": [8]})
dg = DataGenerator(x_train, y_train)
train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test, batch_size=batch_size)

In [ ]:
# started training
mtl_trainer = MTLTrainer(model, task_types=[task_type], optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, n_epoch=epoch, earlystop_patience=50, device=device, model_path=save_dir)
file_path = 'path/to/save/output/result_{}_{}.txt'.format(model_name, task_select)
mtl_trainer.fit(train_dataloader, val_dataloader, mode = 'mark1', seed = 'mark2', file_path = file_path)

In [ ]:
auc = mtl_trainer.evaluate(mtl_trainer.model, test_dataloader)
print(f'test auc: {auc}')

epo = ['test_result']
my_list = epo + auc
my_list = ', '.join(map(str, my_list))

try:
    with open(file_path, 'a') as file:
        file.write(my_list + '\n')
except Exception as e:  
    print(f"An error occurred while adding to the file: {e}")